# Final Project Code

In [1]:
# run in base directory
import os
os.chdir("/home/awesome")
print(os.getcwd())

# datatypes
import json
import yaml
# database connection
import psycopg2
import psycopg2.extras
import psycopg2.extensions as psql_ext
from psycopg2 import sql
# custom etl functions
from etl_process import etl
# respective datasets
from etl_process import station_info as info
# computation
import pandas as pd
# utilities
from pathlib import Path
import itertools
# typing
from typing import Union

# set up directories
HOME_PATH = Path.cwd()
DATA_PATH = HOME_PATH / 'etl_process' / 'processed_data'
SCHEMAS_PATH = HOME_PATH / 'etl_process' / 'schemas'

PROJECT_SCHEMA = 'citibike_project'

/home/awesome


# Set Up Database

### Connect, Set up schema

In [2]:
# PSQL db connection using psycopg2
conn = psycopg2.connect(
    dbname='new_db', 
    user='awesome_user', 
    password='awesome_password', 
    host='postgres', 
    port='5432'
)

In [3]:
etl.drop_recreate_schema(conn, PROJECT_SCHEMA)

Table 'station_info' dropped.
Table 'weather_general' dropped.
Table 'weather_wind' dropped.
Table 'weather_pressure' dropped.
Table 'weather_precip' dropped.
Table 'weather_sky_coverage' dropped.
Table 'irs_codes' dropped.
Table 'nyc_irs' dropped.
Table 'weather_wxcode' dropped.
All tables in citibike_project dropped successfully.
Dropped Schema citibike_project.
Created Schema citibike_project.


### Clean all tables

In [4]:
schema_files = [item for item in SCHEMAS_PATH.iterdir() if item.is_file()]
tables_schemas = list(itertools.chain(*[etl.read_yaml_to_dict(schema_file)["tables"] for schema_file in schema_files]))
tables_schemas = {k: v for d in tables_schemas for k, v in d.items()}

for table_name, table_schema in tables_schemas.items():
    etl.drop_recreate_table(
        db_schema=PROJECT_SCHEMA,
        table_name=table_name,
        table_schema=table_schema,
        conn=conn,
    )


Dropping citibike_project.irs_codes
Creating citibike_project.irs_codes
Dropping citibike_project.nyc_irs
Creating citibike_project.nyc_irs
Dropping citibike_project.station_info
Creating citibike_project.station_info
Dropping citibike_project.weather_general
Creating citibike_project.weather_general
Dropping citibike_project.weather_precip
Creating citibike_project.weather_precip
Dropping citibike_project.weather_wind
Creating citibike_project.weather_wind
Dropping citibike_project.weather_pressure
Creating citibike_project.weather_pressure
Dropping citibike_project.weather_wxcode
Creating citibike_project.weather_wxcode
Dropping citibike_project.weather_sky_coverage
Creating citibike_project.weather_sky_coverage


# Upload Data

### Aggregated Ride Data

In [5]:
# TKTK

### Weather Data

In [5]:
for file in [
    "weather_general",
    "weather_precip",
    "weather_pressure",
    "weather_sky_coverage",
    "weather_wind",
    "weather_wxcode"
]:
    df = pd.read_csv(DATA_PATH / "weather" / f"{file}.csv")
    etl.upload_dataframe(
        conn=conn,
        dataframe=df,
        db_schema=PROJECT_SCHEMA,
        table_name=file,
        table_schema=tables_schemas[file]
    )

Uploaded 11272 records to citibike_project.weather_general
Uploaded 11272 records to citibike_project.weather_precip
Uploaded 11272 records to citibike_project.weather_pressure
Uploaded 11272 records to citibike_project.weather_sky_coverage
Uploaded 11272 records to citibike_project.weather_wind
Uploaded 32 records to citibike_project.weather_wxcode


### Station Info

In [6]:
df_station_status = info.get_station_info_data()

etl.upload_dataframe(
    conn=conn,
    dataframe=df_station_status,
    db_schema=PROJECT_SCHEMA,
    table_name='station_info',
    table_schema=tables_schemas["station_info"]
)

Uploaded 2221 records to citibike_project.station_info


In [7]:
pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info LIMIT 3", conn)

/tmp/ipykernel_61/1126789902.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info LIMIT 3", conn)


,short_name,station_id,name,lat,lon,region_id,capacity,rental_uris
0,5569.07,7d5fa1f0-4069-4d5d-b735-5d500597394a,W Broadway & Watts St,40.723230,-74.003140,71,51,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',..."
1,4962.02,c1a4d909-0a00-475a-8e82-18ed13a4eb01,Whitehall St & Bridge St,40.703662,-74.013181,71,44,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',..."
2,4325.03,7779e057-b33e-46a7-8a1e-8c28bcd9a558,Thomas S. Boyland St & Macon St,40.684910,-73.914930,71,21,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',..."


In [8]:
pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip LIMIT 3", conn)

/tmp/ipykernel_61/2297728758.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip LIMIT 3", conn)


,valid,date,time,hour,one_hour_precip_amount,wxcode1,wxcode2,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,snowdepth
0,2023-06-30 00:51:00,2023-06-30,00:51:00,0,0.00,HZ,NaN,0,0.0,0,0.0
1,2023-06-30 01:51:00,2023-06-30,01:51:00,1,0.00,HZ,NaN,0,0.0,0,0.0
2,2023-06-30 02:51:00,2023-06-30,02:51:00,2,0.00,HZ,NaN,0,0.0,0,0.0


### IRS Data

In [61]:
for file in [
    "irs_codes",
    "nyc_irs",
]:
    df = pd.read_csv(DATA_PATH / "irs" / f"{file}.csv")
    etl.upload_dataframe(
        conn=conn,
        dataframe=df,
        db_schema=PROJECT_SCHEMA,
        table_name=file,
        table_schema=tables_schemas[file]
    )

Uploaded 167 records to citibike_project.irs_codes
Uploaded 1050 records to citibike_project.nyc_irs


### Close the connection

In [62]:
# conn.close()

# Report Generation

In [11]:
df_stations = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info", conn)
df_largest_stations = df_stations.sort_values(by="capacity", ascending=False).iloc[:50]

/tmp/ipykernel_210/137927088.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stations = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info", conn)


In [12]:
df_weather_precip = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip", conn)
df_weather_precip["one_hour_precip_amount"] = pd.to_numeric(df_weather_precip['one_hour_precip_amount'], errors='coerce')
df_weather_precip_daily = df_weather_precip.groupby(by="date")[["one_hour_precip_amount"]].sum()

/tmp/ipykernel_210/808375836.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_weather_precip = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip", conn)


In [13]:
import plotly.graph_objs as go

df_stations = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info", conn)

df_weather_precip = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip", conn)
df_weather_precip["one_hour_precip_amount"] = pd.to_numeric(df_weather_precip['one_hour_precip_amount'], errors='coerce')
df_weather_precip_daily = df_weather_precip.groupby(by="date")[["one_hour_precip_amount"]].sum()

# Create simple plots
bar1 = go.Bar(x=df_weather_precip_daily.index, y=df_weather_precip_daily.one_hour_precip_amount)
layout1 = go.Layout()
fig1 = go.Figure([bar1], layout1)

# Create another simple plot
# Create a table
fig2 = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_largest_stations.to_dict().keys()),
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=df_largest_stations.values.T,
        fill_color='lavender',
        align='left'
    ))
])

# Convert figures to HTML strings
fig1_html = fig1.to_html(full_html=False, include_plotlyjs='cdn')
fig2_html = fig2.to_html(full_html=False, include_plotlyjs='cdn')

# Create the HTML template
template = """
<html>

df_largest_stations = df_stations.sort_values(by="capacity", ascending=False).iloc[:50]
<head>
    <title>Plotly Report</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>CitiBike Report</h1>
    <div id='divPlotly1'>
        <h2>Daily Precipitation</h2>
        {fig1_html}
    </div>
    <div id='divPlotly2'>
        <h2>Largest Stations</h2>
        {fig2_html}
    </div>
</body>
</html>
"""

# Write the HTML report to a file
with open('report.html', 'w') as f:
    f.write(template.format(fig1_html=fig1_html, fig2_html=fig2_html))


/tmp/ipykernel_210/580007286.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stations = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.station_info", conn)
/tmp/ipykernel_210/580007286.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_weather_precip = pd.read_sql_query(f"SELECT * FROM {PROJECT_SCHEMA}.weather_precip", conn)
